## Chained Function Calling

In [1]:
import os
from dotenv import load_dotenv  

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

MODEL="gpt-4.1-mini"

In [ ]:
%pip install -q jsonref

In [2]:
import json
import jsonref 
import requests
from pprint import pp 

from openai import OpenAI

client = OpenAI()

## OpenAPI Specification를 함수 정의로 변환

In [3]:
# 예제 OpenAPI 명세 파일을 읽어온다.
with open('../dataset/example_events_openapi.json', 'r') as f:
    # 아래에서 설명하겠지만, jsonref로 로드하는 것이 중요하다.
    # jsonref.loads는 파일 내의 "$ref" 같은 참조를 실제 객체로 자동 변환해준다.
    openapi_spec = jsonref.loads(f.read())

# 로드된 명세 내용을 화면에 표시한다.
display(openapi_spec)

{'openapi': '3.0.0',
 'info': {'version': '1.0.0',
  'title': 'Event Management API',
  'description': 'An API for managing event data'},
 'paths': {'/events': {'get': {'summary': 'List all events',
    'operationId': 'listEvents',
    'responses': {'200': {'description': 'A list of events',
      'content': {'application/json': {'schema': {'type': 'array',
         'items': {'type': 'object',
          'properties': {'id': {'type': 'string'},
           'name': {'type': 'string'},
           'date': {'type': 'string', 'format': 'date-time'},
           'location': {'type': 'string'}},
          'required': ['name', 'date', 'location']}}}}}}},
   'post': {'summary': 'Create a new event',
    'operationId': 'createEvent',
    'requestBody': {'required': True,
     'content': {'application/json': {'schema': {'type': 'object',
        'properties': {'id': {'type': 'string'},
         'name': {'type': 'string'},
         'date': {'type': 'string', 'format': 'date-time'},
         'location

## OpenAPI Specification을 Function Calling으로 변환

각 함수가 다음 키를 포함하는 딕셔너리로 표현되는 정의 리스트를 생성하기 위해 간단한 `openapi_to_functions` 함수를 작성할 수 있다:

  - `name`: OpenAPI 명세에 정의된 API 엔드포인트의 `operationId`에 해당한다.
  - `description`: 함수가 무엇을 하는지에 대한 개요를 제공하는 간략한 설명 또는 요약이다.
  - `parameters`: 함수에 대한 예상 입력 매개변수를 정의하는 스키마이다. 각 매개변수의 유형, 필수 여부 및 기타 관련 세부 정보에 대한 정보를 제공한다.

스키마에 정의된 각 엔드포인트에 대해 다음을 수행해야 한다:

1.  **JSON 참조 해석**: OpenAPI 명세에서는 중복을 피하기 위해 JSON 참조(또는 $ref)를 사용하는 것이 일반적이다. 이 참조는 여러 곳에서 사용되는 정의를 가리킨다. 예를 들어, 여러 API 엔드포인트가 동일한 객체 구조를 반환하는 경우, 해당 구조를 한 번 정의한 다음 필요한 곳 어디에서나 참조할 수 있다. 우리는 이 참조들을 해석하고 가리키는 내용으로 교체해야 한다.

2.  **함수 이름 추출**: 우리는 단순히 `operationId`를 함수 이름으로 사용할 것이다. 대안으로, 엔드포인트 경로와 작업을 함수 이름으로 사용할 수도 있다.

3.  **설명 및 매개변수 추출**: `description`, `summary`, `requestBody`, `parameters` 필드를 순회하여 함수의 설명과 매개변수를 채울 것이다.

In [4]:
def openapi_to_functions(openapi_spec):
    """
    OpenAPI 명세 객체를 받아 Chat Completions API의 함수 정의 리스트로 변환한다.
    """
    functions = [] # 함수 정의를 저장할 리스트

    # 명세의 'paths'에 있는 모든 경로와 메서드를 순회한다.
    for path, methods in openapi_spec["paths"].items():
        for method, spec_with_ref in methods.items():
            # 1. JSON 참조를 해석한다.
            # jsonref.replace_refs를 사용하여 $ref 부분을 실제 객체로 바꾼다.
            spec = jsonref.replace_refs(spec_with_ref)

            # 2. 함수 이름을 추출한다.
            # operationId를 함수 이름으로 사용한다.
            function_name = spec.get("operationId")

            # 3. 설명과 매개변수를 추출한다.
            desc = spec.get("description") or spec.get("summary", "")

            # 함수의 매개변수 스키마를 초기화한다.
            schema = {"type": "object", "properties": {}}

            # 요청 본문(requestBody) 스키마를 처리한다.
            req_body = (
                spec.get("requestBody", {})
                .get("content", {})
                .get("application/json", {})
                .get("schema")
            )
            if req_body:
                schema["properties"]["requestBody"] = req_body

            # URL 경로 또는 쿼리의 매개변수(parameters)를 처리한다.
            params = spec.get("parameters", [])
            if params:
                param_properties = {
                    param["name"]: param["schema"]
                    for param in params
                    if "schema" in param
                }
                schema["properties"]["parameters"] = {
                    "type": "object",
                    "properties": param_properties,
                }

            # 최종 함수 정의를 리스트에 추가한다.
            functions.append(
                {"type": "function", "function": {"name": function_name, "description": desc, "parameters": schema}}
            )

    return functions


# 위에서 정의한 함수를 사용하여 OpenAPI 명세를 함수 리스트로 변환한다.
functions = openapi_to_functions(openapi_spec)

# 변환된 함수 정의를 하나씩 출력하여 확인한다.
for function in functions:
    pp(function)
    print()

{'type': 'function',
 'function': {'name': 'listEvents',
              'description': 'List all events',
              'parameters': {'type': 'object', 'properties': {}}}}

{'type': 'function',
 'function': {'name': 'createEvent',
              'description': 'Create a new event',
              'parameters': {'type': 'object',
                             'properties': {'requestBody': {'type': 'object',
                                                            'properties': {'id': {'type': 'string'},
                                                                           'name': {'type': 'string'},
                                                                           'date': {'type': 'string',
                                                                                    'format': 'date-time'},
                                                                           'location': {'type': 'string'}},
                                                            'required':

## OpenAI로 함수 호출

In [5]:
def get_openai_response(functions, messages):
    return client.chat.completions.create(
        model=MODEL,
        tools=functions,
        tool_choice="auto",
        temperature=0,
        messages=messages,
        # 필요 시 병렬 tool 호출을 막고 싶다면(모델이 지원할 때만):
        # parallel_tool_calls=False,
    )

def process_user_instruction(functions, instruction):
    MAX_CALLS = 5
    messages = [
        {"role": "system", "content": """
당신은 도움이 되는 어시스턴트입니다.
function_call을 사용하여 다음 프롬프트에 응답한 다음, 수행한 작업들을 요약하세요.
사용자 요청이 모호하면 명확화를 위해 질문하세요.
""" },
        {"role": "user", "content": instruction},
    ]

    for step in range(MAX_CALLS):
        response = get_openai_response(functions, messages)
        msg = response.choices[0].message

        # 1) tool_calls가 있으면: 모든 tool_call에 대해 'tool' 메시지를 각각 추가
        if msg.tool_calls:
            print(f"\n>> 함수 호출 턴 #{step + 1}\n")
            pp(msg.tool_calls)

            # assistant 메시지를 dict로 명시적으로 다시 추가(안정성↑)
            messages.append({
                "role": "assistant",
                "content": msg.content,  # 보통 None이거나 요약 텍스트
                "tool_calls": [
                    {
                        "id": tc.id,
                        "type": "function",
                        "function": {
                            "name": tc.function.name,
                            "arguments": tc.function.arguments,
                        },
                    }
                    for tc in msg.tool_calls
                ],
            })

            # 각 tool_call 별로 결과 메시지 추가 (여기서는 더미 실행)
            for tc in msg.tool_calls:
                fn_name = tc.function.name
                # 모델이 넘긴 인자를 JSON으로 파싱 (빈 문자열/None 방어)
                try:
                    fn_args = json.loads(tc.function.arguments or "{}")
                except Exception:
                    fn_args = {"_raw": tc.function.arguments}

                # 실제로는 여기서 해당 함수 실행 → 결과(result)를 얻어야 함
                fake_result = {
                    "ok": True,
                    "called": fn_name,
                    "args": fn_args,
                    "note": "이 부분에 실제 함수 실행 결과를 넣으세요.",
                }

                messages.append({
                    "role": "tool",
                    "tool_call_id": tc.id,
                    "content": json.dumps(fake_result, ensure_ascii=False),
                })

            # 모든 tool_call에 대해 응답을 붙였으니 다음 루프로 계속
            continue

        # 2) tool_calls가 없으면: 모델이 최종 텍스트를 반환한 것 → 출력 후 종료
        print("\n>> 메시지:\n")
        print(msg.content or "(빈 응답)")
        break
    else:
        print(f"최대 연쇄 함수 호출 횟수에 도달했습니다: {MAX_CALLS}")

# 실행
USER_INSTRUCTION = """
지시사항: 모든 이벤트를 가져오세요.
그런 다음 AGI Party라는 이름의 새 이벤트를 만드세요.
그런 다음 id 2456인 이벤트를 삭제하세요.
"""
functions = openapi_to_functions(openapi_spec)
process_user_instruction(functions, USER_INSTRUCTION)


>> 함수 호출 턴 #1

[ChatCompletionMessageToolCall(id='call_cnAC1WZoDNTkd7YtTfkURp67', function=Function(arguments='{}', name='listEvents'), type='function')]

>> 함수 호출 턴 #2

[ChatCompletionMessageToolCall(id='call_xcIHk2V4lA7xpCKXv7iCR1Ry', function=Function(arguments='{"requestBody": {"name": "AGI Party", "date": "2024-12-31T20:00:00", "location": "Seoul"}}', name='createEvent'), type='function'),
 ChatCompletionMessageToolCall(id='call_6vonx5I0m7Bp92HVAuv7F0H3', function=Function(arguments='{"parameters": {"id": "2456"}}', name='deleteEvent'), type='function')]

>> 메시지:

모든 이벤트를 가져왔고, "AGI Party"라는 이름의 새 이벤트를 생성했으며, ID가 2456인 이벤트를 삭제했습니다.
